In [1]:
import numpy as np
import pandas as pd

In [2]:
from subprocess import check_output
print(check_output(["ls", "data"]).decode("utf8"))

data_preprocess.csv
glove.42B.300d.txt
glove.42B.300d.zip
glove.6B.100d.txt
glove.6B.200d.txt
glove.6B.300d.txt
glove.6B.50d.txt
glove.840B.300d.txt
sample_submission.csv
test.csv
test_manual_features.csv
train.csv
train_manual_features.csv



In [3]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
train = train.sample(frac=1)

In [4]:
list_sentences_train = train["comment_text"].fillna("...").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
#list_classes = ["toxic"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("...").values

In [5]:
from keras.preprocessing import text, sequence

max_features = 20000
maxlen = 256

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, Conv1D, Reshape, MaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2

CONV_SIZE = 64
LSTM_SIZE = 64
EMB_SIZE = 64

def get_model_orig():
    embed_size = 64
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', 'binary_crossentropy'])

    return model


def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, EMB_SIZE, embeddings_regularizer=l2(1e-3))(inp)
    #x = Reshape((maxlen, embed_size, 1))(x)
    #x = Bidirectional(LSTM(LSTM_SIZE, return_sequences=True))(x)
    x = Conv1D(CONV_SIZE, 2, strides=1, padding='valid', activation='selu')(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(CONV_SIZE, 2, strides=1, padding='valid', activation='selu')(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(CONV_SIZE, 2, strides=1, padding='valid', activation='selu')(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(CONV_SIZE, 2, strides=1, padding='valid', activation='selu')(x)
    x = MaxPooling1D(2)(x)
    x = Bidirectional(LSTM(LSTM_SIZE))(x)
    #x = Dense(LSTM_SIZE, activation="selu")(x)
    x = Dense(len(list_classes), activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', 'binary_crossentropy'])

    return model

model = get_model_orig()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 256, 64)           1280000   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 1,280,390
Trainable params: 1,280,390
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 32
epochs = 20000

file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks_list)

Train on 91058 samples, validate on 4793 samples
Epoch 1/20000
91058/91058 [==============================] - 47s 521us/step - loss: 0.1168 - acc: 0.9663 - binary_crossentropy: 0.1168 - val_loss: 0.0706 - val_acc: 0.9759 - val_binary_crossentropy: 0.0706
Epoch 2/20000
91058/91058 [==============================] - 46s 503us/step - loss: 0.0603 - acc: 0.9792 - binary_crossentropy: 0.0603 - val_loss: 0.0605 - val_acc: 0.9790 - val_binary_crossentropy: 0.0605
Epoch 3/20000
91058/91058 [==============================] - 45s 491us/step - loss: 0.0534 - acc: 0.9809 - binary_crossentropy: 0.0534 - val_loss: 0.0572 - val_acc: 0.9795 - val_binary_crossentropy: 0.0572
Epoch 4/20000
91058/91058 [==============================] - 42s 463us/step - loss: 0.0498 - acc: 0.9817 - binary_crossentropy: 0.0498 - val_loss: 0.0565 - val_acc: 0.9800 - val_binary_crossentropy: 0.0565
Epoch 5/20000
91058/91058 [==============================] - 43s 468us/step - loss: 0.0474 - acc: 0.9825 - binary_crossentropy:

In [21]:
model.load_weights(file_path)

y_test = model.predict(X_te)



sample_submission = pd.read_csv("./data/sample_submission.csv")

sample_submission[list_classes] = y_test



sample_submission.to_csv("baseline.csv", index=False)